In [ ]:
from datasets import load_dataset

ds = load_dataset("hf-vision/chest-xray-pneumonia")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00007.parquet:   0%|          | 0.00/446M [00:00<?, ?B/s]

data/train-00001-of-00007.parquet:   0%|          | 0.00/385M [00:00<?, ?B/s]

data/train-00002-of-00007.parquet:   0%|          | 0.00/68.9M [00:00<?, ?B/s]

...

data/validation-00000-of-00001.parquet:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/78.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5216 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/16 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/624 [00:00<?, ? examples/s]

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models

In [ ]:

IMG_SIZE = 224
BATCH_SIZE = 32

def preprocess(example):
    '''
    Preprocess the PIL image (python image library) into array numbers 
    '''
    image = example["image"].convert("L")  # grayscale
    image = image.resize((IMG_SIZE, IMG_SIZE))
    image = np.array(image, dtype=np.float32) / 255.0
    image = np.expand_dims(image, axis=-1)  # (224,224,1)
    label = example["label"]
    return image, label

In [ ]:
def hf_to_tf(dataset):
    '''
    it converts the hugging face datasets to tensor flow training pipeline
    '''
    return tf.data.Dataset.from_generator( # creates an tensorflow datasets by pulling dataa one item at a time in pythin generator
        lambda: (preprocess(x) for x in dataset),
        output_signature=(
            tf.TensorSpec(shape=(224,224,1), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int64)
        ) # output signature just gives the shapes and type of data that matches the resnet18 architecture
    )

train_ds = hf_to_tf(ds["train"]).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
# prefech:- it prepares the next batch while current batch being trained
# shuffle:- it mixes the training images randomly before feeding them to the model.
val_ds   = hf_to_tf(ds["validation"]).batch(BATCH_SIZE)
test_ds  = hf_to_tf(ds["test"]).batch(BATCH_SIZE)
#test

In [ ]:
def residual_block(x, filters, stride=1):
    shortcut = x

    x = layers.Conv2D(filters, 3, strides=stride, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(filters, 3, strides=1, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, 1, strides=stride, use_bias=False)(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)

    return x


In [ ]:
def ResNet18(input_shape=(224,224,1), num_classes=2):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(64, 7, strides=2, padding="same", use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

    # Conv2_x
    x = residual_block(x, 64)
    x = residual_block(x, 64)

    # Conv3_x
    x = residual_block(x, 128, stride=2)
    x = residual_block(x, 128)

    # Conv4_x
    x = residual_block(x, 256, stride=2)
    x = residual_block(x, 256)

    # Conv5_x
    x = residual_block(x, 512, stride=2)
    x = residual_block(x, 512)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return models.Model(inputs, outputs)
